In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd

# 6.2 Measurement of current-voltage characteristic

In [ ]:
U, I = np.genfromtxt("data/IV.txt",unpack=True)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax.plot(U, I, "b-", label="Current-Voltage Characteristic for reverse bias")

ax.set_xlabel("Bias Voltage $U / V$")
ax.set_ylabel("Leakage Current $I / \mu A$")

ax.legend(loc="best")
ax.grid()

# 6.3 Pedestals and noise

## a) Plot an overview of the pedestals and noise for each strip

In [ ]:
# Import matrix of events and channels where axis 0 represents the channels, axis 1 the events
ADC_ik = np.genfromtxt("data/Pedestal.txt",delimiter=";")

# Calculate pedestals (should be ~500)
P = ADC_ik.sum(axis=1)/ADC_ik.shape[1]

fig, ax = plt.subplots(constrained_layout=True)

ax.plot(P, "b-", label="Pedestals per Channel")

ax.set_xlabel("ADC Channel")
ax.set_ylabel("ADC Counts")

ax.legend(loc="best")
ax.grid()

In [ ]:
# Calculate Common Mode Shift (should be gaussian distributed around 0)
D = (ADC_ik.T - P).T.sum(axis=0)/len(P)

plt.hist(D)

In [ ]:
# Calculate Noise per channel from ADC counts, Pedestals and CMS
Noise = np.sqrt( (( (ADC_ik.T - P).T - D)**2).sum(axis=1) / (len(D)-1))

fig, ax = plt.subplots(constrained_layout=True)

plt.plot(Noise, "b-", label="Noise per channel")

ax.set_xlabel("ADC Channel")
ax.set_ylabel("Noise")

ax.legend(loc="best")
ax.grid()

## b) Graphically represent the values of the Common Mode in a meaningful way

In [ ]:
bins_D = np.linspace(-7.5,7.5,16)

fig, ax = plt.subplots(constrained_layout=True)

plt.hist(D,bins=bins_D,label="Distribution of Common Mode Shift")

ax.set_xlabel("Common Mode Shift $D$")
ax.set_ylabel("Counts")

ax.legend(loc="best")

# 6.4 Calibration measurements

## a) Plot the measured values of the _Calibration Runs_ and their mean values

In [ ]:
# Import all pulse charges/determined energies
# (Also apply fix to convert commas as decimal delimiter to dots, what idiot wrote the program that saves the data like that???)
def conv(x):
    return x.replace(',', '.').encode()

pulse_charge, counts_C20 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C20.txt")),unpack=True,skip_header=2)
counts_C30 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C30.txt")),usecols=1,skip_header=2)
counts_C40 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C40.txt")),usecols=1,skip_header=2)
counts_C50 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C50.txt")),usecols=1,skip_header=2)
counts_C60 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C60.txt")),usecols=1,skip_header=2)
counts_0VC20 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C20_0V.txt")),usecols=1,skip_header=2)

In [ ]:
counts_mean = (counts_C20+counts_C30+counts_C40+counts_C50+counts_C60)/5

plt.plot(counts_C20, pulse_charge, "x")
plt.plot(counts_C30, pulse_charge, ".")
plt.plot(counts_C40, pulse_charge, "o")
plt.plot(counts_C50, pulse_charge, "v")
plt.plot(counts_C60, pulse_charge, "+")
plt.plot(counts_0VC20, pulse_charge, "x")
plt.plot(counts_mean, pulse_charge, "b-")
plt.plot()

## b) Determine the dependene of the injected charge on the measured ADC values with a 4th degree polynomial

In [ ]:
pulse_energy = 3.6*pulse_charge/1000
poly_par, poly_cov = np.polyfit(counts_mean[pulse_charge<50000],pulse_energy[pulse_charge<50000],4,cov=True)


poly = np.poly1d(poly_par)

x = np.linspace(0,270,10000)


fig, ax = plt.subplots(constrained_layout=True)

ax.plot(counts_mean,pulse_energy, "b-", label="Injected pulse energy for measured ADC counts")
ax.plot(x,poly(x), "r-", label="Polynomial fit (4th degree)")

ax.set_xlabel("ADC Counts")
ax.set_ylabel("Energy of injected Charge $E / keV$")

ax.legend(loc="best")
ax.grid()

## c) Compare the curve at 0 V and above the depletion voltage

In [ ]:
poly_par_0V, poly_cov_0V = np.polyfit(pulse_charge,counts_0VC20,4,cov=True)
poly_0V = np.poly1d(poly_par_0V)

plt.plot(pulse_charge,counts_mean,".")
plt.plot(x,poly(x))
plt.plot(pulse_charge,counts_0VC20,".")
plt.plot(x,poly_0V(x))

# 6.5 Measuring the strip sensors by using a laser

## c) Plot the signal of the relevant strips as a function of the laser position. Determine from this the _pitch_ of the strips, the extension of the laser on it and note the strip numbers

In [ ]:
A = np.genfromtxt("data/Laserscan.txt")
A.shape

In [ ]:
plt.matshow(A)
plt.colorbar()
strip_no = np.argwhere(A.sum(axis=0)>150)

In [ ]:
x_laser = np.arange(0,350,10)
C_laser = A.sum(axis=1)
max_tot = C_laser>175
d_strip = np.diff(x_laser[max_tot]).mean()

plt.plot(x_laser, C_laser)
plt.plot(x_laser, C_laser,".")
plt.plot(x_laser[max_tot],C_laser[max_tot],"x")
d_strip

In [ ]:
C_laser_s = A[:,strip_no]

markers = ["-","--","-.",":"]

for i in range(4):
    plt.plot(x_laser,C_laser_s[:,i],markers[i])

# 6.6 Determination of the Charge Colelction Efficiency

## Using a laser

### a) Investigate the efficiency of the detector as a function of the applied voltage by measuring one of the maxima from task 6.5 at different bias voltages with the laser in focus. Compare the beginning of the pleateau with the depletion voltage determined in task 6.2

In [ ]:
CCEL = np.empty((128,21))
for i in range(21):
    CCEL[:,i] = np.genfromtxt(f"data/CCEL/{i*10}VCCEL.txt")

In [ ]:
CCEL_mean = CCEL[76:78].mean(axis=0)
plt.plot(U,CCEL[77])
plt.plot(U,100*I/np.amax(I))

In [ ]:
def dc(U):
    d_c = np.empty(len(U))
    for i in range(len(U)):
        if U[i] < U_dep:
            d_c[i] = D * np.sqrt(U[i]/U_dep)
        else:
            d_c[i] = D
    return d_c

def CCE_U(U,a,b):
    CCE_U = (1 - np.exp(-dc(U+b)/a)) / (1 - np.exp(-D/a))
    
    return CCE_U

U_dep = 70
D = 300e-6

CCEL_par, CCEL_cov = curve_fit(CCE_U, U[U<U_dep], CCEL_mean[U<U_dep]/100, p0=[1e-5,1])
print(CCEL_par)

In [ ]:
U_fit = np.linspace(0,200,100)
plt.plot(U_fit, CCE_U(U_fit,*CCEL_par)*100)
plt.plot(U[U<U_dep],CCEL_mean[U<U_dep])

## Using a $\beta^-$-source

### a) Plot the mean cluster energy as a function of the applied voltage

### b) Compare the results between the CCE measurement with a laser and the source. Why do they differ?

In [ ]:
names = [f"c{i}" for i in range(13)]
names2 = [f"{i*10}V" for i in range(21)]

df_CCEQ = pd.concat([pd.read_csv(f"data/CCEQ/{i*10}V_Cluster_adc_entries.txt",delimiter="\t",names=names).fillna(0).sum(axis=1) for i in range(21)],axis=1,ignore_index=True,names=names2).fillna(0)
df_CCEQ

In [ ]:
CCEQ = df_CCEQ.to_numpy().T
CCEQ_mean = CCEQ.mean(axis=1)
plt.plot(U,CCEQ_mean)
plt.plot(U,CCEL_mean)

# 6.7 Large source scan

## a) Represent the clusters per event and the channels per cluster in a meaningful way.

In [ ]:
no_clusters = np.genfromtxt("data/number_of_clusters.txt")
size_clusters = np.genfromtxt("data/cluster_size.txt")

channels = np.arange(0.5,129.5,1)

fig, ax = plt.subplots(constrained_layout=True)

ax.stairs(no_clusters, channels, label="Clusters per Event")
ax.stairs(size_clusters, channels, label="Channels per cluster")

ax.set_xlabel("# of clusters per event or channels per cluster")
ax.set_ylabel("Counts")

ax.set_yscale("symlog")
ax.set_xlim((-1,20))

ax.legend(loc="best")
ax.grid()

## b) Display the number of events per channel

In [ ]:
hitmap = np.genfromtxt("data/hitmap.txt")


fig, ax = plt.subplots(constrained_layout=True)

ax.stairs(hitmap,channels,label="Hitmap of events per ADC channel")

ax.set_xlabel("ADC Channel")
ax.set_ylabel("Counts")

ax.legend(loc="lower right")
ax.grid()

## c) Plot the energy spectrum in ADC values and in keV

In [ ]:
C_ADC = np.genfromtxt("data/cluster_ADCs.txt")

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax.hist(C_ADC,bins=20, label="Spectrum of ADC values for source scan")
ax.set_yscale("log")

ax.set_xlabel("ADC Values")
ax.set_ylabel("Counts")

ax.legend(loc="best")
ax.grid()

In [ ]:
C_E = poly(C_ADC)

logx = np.logspace(np.log10(np.amin(C_E)),np.log10(np.amax(C_E)+1),51)

hist_E, bins_E = np.histogram(C_E,bins=logx)

E_mean = C_E.mean()
E_MPV = (bins_E[np.argmax(hist_E)+1] + bins_E[np.argmax(hist_E)])/2


fig, ax = plt.subplots(constrained_layout=True)

ax.stairs(hist_E, bins_E, label="Energy spectrum for source scan")
ax.axvline(E_mean, color="r", linestyle="--", label="Mean energy value")
ax.axvline(E_MPV, color="g", linestyle="--", label="Most probable energy value")

ax.set_xlabel("Energy $E/keV$")
ax.set_ylabel("Counts")
ax.set_xscale("log")

ax.legend(loc="best")
ax.grid()
print(E_mean, E_MPV)